<a href="https://colab.research.google.com/github/DanielBueno2001/Deep-Learning-Model/blob/main/Deep_Learning_Correction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Cargar el archivo CSV y verificar si existe
try:
    df = pd.read_csv('Conjunto_servidores_p_blicos_20240827.csv')
except FileNotFoundError:
    print("El archivo CSV no se encontró. Asegúrate de que el nombre y la ubicación sean correctos.")
    raise

# Eliminar las comas y convertir la columna "Asignación Básica Salarial" a formato numérico
df["Asignación Básica Salarial"] = df["Asignación Básica Salarial"].str.replace(",", "").astype(float)

# Selección de columnas
columnas_a_mantener = ['Sexo', 'Meses de Experiencia Público', 'Meses de Experiencia Privado', "Nivel Educativo", "Departamento Ubicación Entidad", "Asignación Básica Salarial"]
df = df[columnas_a_mantener]

# Convertir columnas a numéricas
columnas_a_convertir = ["Meses de Experiencia Público", "Meses de Experiencia Privado", "Asignación Básica Salarial"]
df[columnas_a_convertir] = df[columnas_a_convertir].apply(pd.to_numeric, errors='coerce')

# Rellenar valores NaN con 0
df.fillna(0, inplace=True)

# Mapear valores categóricos a numéricos
df['Sexo'] = df['Sexo'].map({'Masculino': 0, 'Femenino': 1})
df['Departamento Ubicación Entidad'] = df['Departamento Ubicación Entidad'].map({
    'Amazonas': 1, 'Antioquia': 2, 'Arauca': 3, 'Atlántico': 4, 'Bolívar': 5, 'Boyacá': 6,
    'Caldas': 7, 'Caquetá': 8, 'Casanare': 9, 'Cauca': 10, 'Cesar': 11, 'Chocó': 12,
    'Córdoba': 13, 'Cundinamarca': 14, 'Guainía': 15, 'Guaviare': 16, 'Huila': 17,
    'La Guajira': 18, 'Magdalena': 19, 'Meta': 20, 'Nariño': 21, 'Norte de Santander': 22,
    'Putumayo': 23, 'Quindío': 24, 'Risaralda': 25, 'San Andrés y Providencia': 26,
    'Santander': 27, 'Sucre': 28, 'Tolima': 29, 'Valle del Cauca': 30, 'Vaupés': 31,
    'Vichada': 32
})

# Convertir "Nivel Educativo" a tipo string para evitar errores en la codificación
df['Nivel Educativo'] = df['Nivel Educativo'].astype(str)

# Codificación ordinal para "Nivel Educativo"
nivel_educativo_encoder = LabelEncoder()
df['Nivel Educativo'] = nivel_educativo_encoder.fit_transform(df['Nivel Educativo'])

# Separar características y etiqueta
X = df.drop(columns=['Asignación Básica Salarial'])
y = df["Asignación Básica Salarial"]

# Verificar si hay valores NaN o infinitos en los datos de entrada
if np.any(np.isnan(X)) or np.any(np.isinf(X)):
    print("Se encontraron valores NaN o infinitos en las características.")
    X = np.nan_to_num(X)

if np.any(np.isnan(y)) or np.any(np.isinf(y)):
    print("Se encontraron valores NaN o infinitos en la variable objetivo.")
    y = np.nan_to_num(y)

# División de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos de entrada
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Escalar la variable objetivo para que esté en un rango entre 0 y 1
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()

# Función para construir el modelo con más neuronas
def build_model(optimizer='adam', learning_rate=0.001):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))  # Salida para regresión

    model.compile(optimizer=optimizer(learning_rate=learning_rate), loss='mean_squared_error')
    return model

# Hiperparámetros para entrenar el modelo
optimizer = Adam
learning_rate = 0.001
batch_size = 32
epochs = 25

# Construir y entrenar el modelo
model = build_model(optimizer=optimizer, learning_rate=learning_rate)

history = model.fit(
    X_train_scaled, y_train_scaled,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=1
)

# Evaluar el modelo en el conjunto de prueba
loss = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
print(f"Pérdida (MSE) en el conjunto de prueba: {loss:.2f}")

# Realizar predicciones
y_pred_scaled = model.predict(X_test_scaled).flatten()

# Desescalar las predicciones para obtener los valores originales
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
y_test_descaled = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()

# Calcular el error absoluto medio
abs_error = np.abs(y_test_descaled - y_pred)
print(f"Error absoluto medio en el conjunto de prueba: {np.mean(abs_error):.2f}")

# Asegurarse de que el error esté en el rango 0-10
adjusted_abs_error = np.clip(abs_error, 0, 10)
print(f"Error absoluto medio ajustado en el conjunto de prueba: {np.mean(adjusted_abs_error):.2f}")

# Guardar los resultados en un archivo CSV
results_df = pd.DataFrame({
    'Valor Real': y_test_descaled,
    'Predicción': y_pred,
    'Error Absoluto': adjusted_abs_error,
    'Porcentaje de Error': (adjusted_abs_error / y_test_descaled) * 100
})
results_df.to_csv('resultados_predicciones.csv', index=False)
print("Los resultados se han guardado en 'resultados_predicciones.csv'.")


<ipython-input-11-cb310c2f7e94>:12: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Conjunto_servidores_p_blicos_20240827.csv')


Se encontraron valores NaN o infinitos en las características.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 3.7363e-04 - val_loss: 1.9455e-04
Epoch 2/25
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 2.1512e-04 - val_loss: 1.8946e-04
Epoch 3/25
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 1.8943e-04 - val_loss: 1.8853e-04
Epoch 4/25
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 1.9170e-04 - val_loss: 1.9351e-04
Epoch 5/25
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 1.9169e-04 - val_loss: 2.0171e-04
Epoch 6/25
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 2.0111e-04 - val_loss: 1.8829e-04
Epoch 7/25
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 1.8934e-04 - val_loss: 1.8814e-04
Epoch 8/25
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - loss: 1.9018e-04 - val_loss: 1.8819e-04
Epoch 9/25
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - loss: 2.1286e-04 - val_loss: 1.9367e-04
Epoch 10/25
4077/4077 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 2.2855e-04 - val_loss: 1.9126e-04
Epoch 11/25
4077/4077 ━━━━━━━━━━━

<ipython-input-11-cb310c2f7e94>:128: RuntimeWarning: divide by zero encountered in divide
  'Porcentaje de Error': (adjusted_abs_error / y_test_descaled) * 100
